In [19]:
import pandas as pd

In [8]:
!uv add pandasql

Resolved 121 packages in 4.45s                                       ⠋ Resolving dependencies...                                                     
   Building pandasql==0.7.3                                            
   Building pandasql==0.7.3                                    
      Built pandasql==0.7.3                                    
Prepared 1 package in 412ms                                              
░░░░░░░░░░░░░░░░░░░░ [0/1] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 8ms                                  
 + pandasql==0.7.3


In [9]:
from pandasql import sqldf

In [4]:
df = pd.read_parquet('green_tripdata_2025-11.parquet')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               46912 non-null  int32         
 1   lpep_pickup_datetime   46912 non-null  datetime64[us]
 2   lpep_dropoff_datetime  46912 non-null  datetime64[us]
 3   store_and_fwd_flag     41343 non-null  object        
 4   RatecodeID             41343 non-null  float64       
 5   PULocationID           46912 non-null  int32         
 6   DOLocationID           46912 non-null  int32         
 7   passenger_count        41343 non-null  float64       
 8   trip_distance          46912 non-null  float64       
 9   fare_amount            46912 non-null  float64       
 10  extra                  46912 non-null  float64       
 11  mta_tax                46912 non-null  float64       
 12  tip_amount             46912 non-null  float64       
 13  t

In [16]:
query = """
SELECT COUNT(*) AS total_trips
FROM df
WHERE lpep_pickup_datetime >= '2025-11-01' AND lpep_pickup_datetime < '2025-12-01' AND trip_distance <= 1
"""
result  = sqldf(query,locals())
result

,total_trips
0,8007


In [18]:
query = """
SELECT lpep_pickup_datetime, MAX(trip_distance) AS max_trip_distance
FROM df
WHERE trip_distance < 100
GROUP BY lpep_pickup_datetime
ORDER BY max_trip_distance DESC
LIMIT 1
"""
result  = sqldf(query,locals())
result

,lpep_pickup_datetime,max_trip_distance
0,2025-11-14 15:36:27.000000,88.03


In [20]:
taxizone = pd.read_csv('taxi_zone_lookup.csv')

In [23]:
query = """
SELECT SUM(total_amount) AS highest_fare, Zone
FROM df join taxizone
ON df.PULocationID = taxizone.LocationID
WHERE DATE(lpep_pickup_datetime) = '2025-11-18'
GROUP BY Zone
ORDER BY highest_fare DESC
LIMIT 1
"""
result  = sqldf(query,locals())
result

,highest_fare,Zone
0,9281.92,East Harlem North


In [28]:
query = """
SELECT 
    dropoff_zone.Zone AS dropoff_zone,
    MAX(tip_amount) AS largest_tip
FROM df
JOIN taxizone AS pickup_zone 
    ON df.PULocationID = pickup_zone.LocationID
JOIN taxizone AS dropoff_zone 
    ON df.DOLocationID = dropoff_zone.LocationID
WHERE pickup_zone.Zone = 'East Harlem North'
  AND lpep_pickup_datetime >= '2025-11-01 00:00:00'
  AND lpep_pickup_datetime < '2025-12-01 00:00:00'
GROUP BY dropoff_zone.Zone
ORDER BY largest_tip DESC
LIMIT 1
"""
result  = sqldf(query,locals())
result

,dropoff_zone,largest_tip
0,Yorkville West,81.89
